In [1]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [2]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [3]:
soup_corona = bs(html_corona, 'html.parser')

In [4]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [5]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
#replace items using wild card and regex to have that applied
corona_df["NewCases"].replace("\,","",regex = True, inplace = True)
corona_df["NewCases"].replace("\+","", regex = True, inplace = True)
corona_df["NewCases"] = corona_df["NewCases"].astype("float")
corona_df["NewDeaths"].replace("\,","",regex = True, inplace = True)
corona_df["NewDeaths"].replace("\+","", regex = True, inplace = True)
corona_df["NewDeaths"] = corona_df["NewDeaths"].astype("float")


#add date column for date data scrped
corona_df["Date"]= dt.datetime.today()

In [6]:
corona_df

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/ 1M pop,Population,Date
0,0.0,World,5090157,6746.0,329739.0,500.0,2024329.0,2736089.0,45796.0,653.0,42.3,0.0,0.0,0.000000e+00,2020-05-21 01:52:27.913081
1,1.0,USA,1593039,316.0,94941.0,5.0,370812.0,1127286.0,17815.0,4816.0,287.0,14173807.0,42849.0,3.307852e+08,2020-05-21 01:52:27.913081
2,2.0,Russia,308705,0.0,2972.0,0.0,85392.0,220341.0,2300.0,2115.0,20.0,7578029.0,51930.0,1.459276e+08,2020-05-21 01:52:27.913081
3,3.0,Brazil,293357,0.0,18894.0,0.0,116683.0,157780.0,8318.0,1381.0,89.0,735224.0,3462.0,2.123892e+08,2020-05-21 01:52:27.913081
4,4.0,Spain,279524,0.0,27888.0,0.0,196958.0,54678.0,1152.0,5979.0,596.0,3037840.0,64977.0,4.675280e+07,2020-05-21 01:52:27.913081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,211.0,Western Sahara,6,0.0,0.0,0.0,6.0,0.0,0.0,10.0,0.0,0.0,0.0,5.955430e+05,2020-05-21 01:52:27.913081
212,212.0,Anguilla,3,0.0,0.0,0.0,3.0,0.0,0.0,200.0,0.0,0.0,0.0,1.498800e+04,2020-05-21 01:52:27.913081
213,213.0,Lesotho,1,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,0.0,2.140328e+06,2020-05-21 01:52:27.913081
214,214.0,Saint Pierre Miquelon,1,0.0,0.0,0.0,1.0,0.0,0.0,173.0,0.0,0.0,0.0,5.797000e+03,2020-05-21 01:52:27.913081


In [7]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)
#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)
df_used["ClosedCases"] = df_used["TotalDeaths"] +  df_used["TotalRecovered"]

df_used["ClosedDeathRate"] = df_used["TotalDeaths"]/df_used["ClosedCases"]
df_used["ClosedDeathRate"].fillna(0,inplace = True)

df_used["CurrentDeathRate"] = df_used["TotalDeaths"]/df_used["TotalCases"]
df_used["CurrentDeathRate"].fillna(0,inplace = True)

total_cases = df_used["TotalCases"].sum()
total_closed_cases = df_used["ClosedCases"].sum()

df_used["Weight"] = [x/total_closed_cases for x in df_used["ClosedCases"]]

# #checking if total weight is 1 
# validate_weight = df_used["Weight"].sum()


df_used["DeathRateXWeight"] = df_used["CurrentDeathRate"] *df_used["Weight"]
df_used["DeathRateXWeight"].sum()
df_used["notes"] = ["Last 8 columns are added by Roaya" for x in df_used["DeathRateXWeight"]  ]
df_used.sort_values(by = "DeathRateXWeight", ascending = False)

,#,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,...,Population,Date,Longitude,Latitude,ClosedCases,ClosedDeathRate,CurrentDeathRate,Weight,DeathRateXWeight,notes
0,1.0,USA,1593039,316.0,94941.0,5.0,370812.0,1127286.0,17815.0,4816.0,...,330785250.0,2020-05-21 01:52:27.913081,-97.922211,39.381266,465753.0,0.203844,0.059597,0.198342,0.011821,Last 8 columns are added by Roaya
5,6.0,Italy,227364,0.0,32330.0,0.0,132282.0,62752.0,676.0,3760.0,...,60471440.0,2020-05-21 01:52:27.913081,12.070013,42.796626,164612.0,0.196401,0.142195,0.070100,0.009968,Last 8 columns are added by Roaya
3,4.0,Spain,279524,0.0,27888.0,0.0,196958.0,54678.0,1152.0,5979.0,...,46752802.0,2020-05-21 01:52:27.913081,-3.647550,40.244487,224846.0,0.124032,0.099770,0.095751,0.009553,Last 8 columns are added by Roaya
6,7.0,France,181575,0.0,28132.0,0.0,63354.0,90089.0,1794.0,2782.0,...,65257613.0,2020-05-21 01:52:27.913081,2.618787,47.824905,91486.0,0.307501,0.154933,0.038959,0.006036,Last 8 columns are added by Roaya
2,3.0,Brazil,293357,0.0,18894.0,0.0,116683.0,157780.0,8318.0,1381.0,...,212389176.0,2020-05-21 01:52:27.913081,-53.097831,-10.787777,135577.0,0.139360,0.064406,0.057736,0.003719,Last 8 columns are added by Roaya
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,178.0,Macao,45,0.0,0.0,0.0,45.0,0.0,0.0,69.0,...,648306.0,2020-05-21 01:52:27.913081,113.509321,22.223117,45.0,0.000000,0.000000,0.000019,0.000000,Last 8 columns are added by Roaya
138,154.0,Mozambique,156,0.0,0.0,0.0,48.0,108.0,0.0,5.0,...,31146622.0,2020-05-21 01:52:27.913081,35.533675,-17.273816,48.0,0.000000,0.000000,0.000020,0.000000,Last 8 columns are added by Roaya
141,159.0,Mongolia,140,0.0,0.0,0.0,26.0,114.0,27.0,43.0,...,3272084.0,2020-05-21 01:52:27.913081,103.052998,46.826815,26.0,0.000000,0.000000,0.000011,0.000000,Last 8 columns are added by Roaya
145,163.0,Cambodia,122,0.0,0.0,0.0,122.0,0.0,0.0,7.0,...,16692053.0,2020-05-21 01:52:27.913081,104.906943,12.720048,122.0,0.000000,0.000000,0.000052,0.000000,Last 8 columns are added by Roaya


In [8]:
df_used.to_csv('data.csv')
df_used.to_html('data.html')

In [9]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [10]:
corona_json

'[{"#":1.0,"Country":"USA","TotalCases":1593039,"NewCases":316.0,"TotalDeaths":94941.0,"NewDeaths":5.0,"TotalRecovered":370812.0,"ActiveCases":1127286.0,"CriticalCases":17815.0,"Tot\\u00a0Cases\\/1M pop":4816.0,"Deaths\\/1M pop":287.0,"TotalTests":14173807.0,"Tests\\/ 1M pop":42849.0,"Population":330785250.0,"Date":1590025947913,"Longitude":-97.922211,"Latitude":39.381266,"ClosedCases":465753.0,"ClosedDeathRate":0.2038440976,"CurrentDeathRate":0.059597411,"Weight":0.1983415643,"DeathRateXWeight":0.0118206437,"notes":"Last 8 columns are added by Roaya"},{"#":2.0,"Country":"Russia","TotalCases":308705,"NewCases":0.0,"TotalDeaths":2972.0,"NewDeaths":0.0,"TotalRecovered":85392.0,"ActiveCases":220341.0,"CriticalCases":2300.0,"Tot\\u00a0Cases\\/1M pop":2115.0,"Deaths\\/1M pop":20.0,"TotalTests":7578029.0,"Tests\\/ 1M pop":51930.0,"Population":145927633.0,"Date":1590025947913,"Longitude":96.68656112,"Latitude":61.98052209,"ClosedCases":88364.0,"ClosedDeathRate":0.0336336064,"CurrentDeathRate"